### Nomenclature 
NN      Neural Network  
PINN       Physics-Informed Neural Network
___
___
# Background
On the first day of the experiment, you learned how to implement and train a neural network using the PyTorch Library.
Today, we will apply this knowledge to a more challenging physical problem: the quantum mechanical harmonic oscillator. The task is to train a PINN to predict the time evolution of a system using the initial wavefunction at $t=0$ and the time-dependent Schrodinger equation.
Once model training is complete, we inspect its final predictions to ensure they align with the theory both qualitatively and quantitatively. We then expand the complexity of the physical system to approve the network's abilities.

To emphasize the physical aspects, this template simplifies the machine learning process by providing pre-prepared data, a model blueprint, and the training algorithm. You only need to implement the logic into the loss functions and execute it. Good luck!
___
___
# Tasks

### Implementation 
This experiment focuses on executing the PINN rather than implementing it.
Therefore, large portions of the code are already provided.
First, gain an understanding of the model you are working with, and then complete the implementation.

1. __Understand code__
    1. Visualize imported data.
    2. inspect the `prepare_data` function, the `PINN` class and the `execute` function.
    3. Take notes for future evaluation.
<br><br>

2. __Implement Loss__
    1. Complete the `compute_boundary_loss` function.
    2. Complete the `compute_initial_loss` function.
    3. Complete the `compute_physics_informed_loss` function.
<br><br>
  
3. __Verify functionality__
    1. execute the network training.
    2. Visualize the network prediction using the `AnimationCreator` Class.
    3. Have your tutor approve your work before saving the contour plots for future evaluation.
       Also animate the solution using the Animator.
___
### Execution 

4. __Energy Eigenvalues__ 

    Goal of this Task is, to compute the energy eigenvalue corresponding to the predicted wavefunction.
    <br><br>

5. __Diminishing Potential__  

    Now we want to explore how the wave function will behave in a diminishing potential.  
___
___
__Note__ for evaluation:

When evaluating, it is important to explain the implementation and results obtained.  
Be sure to take notes while completing Task __1__ and save the plots from Tasks __3.3__, __4.4__, and __5.3__.




# Library Import

In [ ]:
import os

# The following line is used to deactivate the use of the GPU.
# If you want to utilize the GPU, just comment the codeline and restart the Kernel.
# The print command in the end of the block states whether CPU or GPU are utilized.

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

from pyDOE import *
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import math 
from tqdm import trange
from torch.autograd import grad
from utils.data_generator import Schrodinger_Boundary, Schrodinger, Schrodinger_Initial, SharedData
from utils.Schrodinger_Animator import AnimationCreator
from utils.tools import SolutionVisualizer, logarithmic_sampling, get_prediction, visualize_training, visualize_imported_data
from matplotlib.ticker import ScalarFormatter

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Torch utilizing dev",device)

Torch utilizing dev cuda:0


# Data Import

The data we are working with describes the initial state of the quantum harmonic oscillator.
It is generated using four imported classes.  
- The `shared_data` class contains all relevant information about the system, such as the length of the time interval `period` and order `n` of the wavefunction.  

- To create data , we use three classes named after their purpose:
    1. The `Schrodinger_Boundary` class provides collocation points at $x\pm 5$. The corresponding loss function is evaluated to induce symmetry to the system.
    2. The `Schrodinger_Initial` class provides labeled data at $t=0$. This data is used to infer the initial condition via the associated loss function into the system.
    3. The `Schrodinger` class provides collocation in the $x,t$-plane, where the physics loss function is computed.

__Tasks__:  
Run the following two cells and become familiar with the provided data sets and take notes.

In [ ]:
shared_data = SharedData(n=0)
boundary = Schrodinger_Boundary(shared_data)
initial = Schrodinger_Initial(shared_data)
schrodinger = Schrodinger(shared_data)
period = shared_data.period
print("Period:",period/math.pi, "pi")


In [ ]:
visualize_imported_data(boundary, initial, schrodinger, shared_data, save_svg=True)

# Data preperation

To prepare for further processing, the datasets mentioned above must be split into two partitions to create a training and testing dataset.

__Task:__  
Review the following function and takes notes. Pay particular attention to:
- its inputs and outputs.
- The entries of keys and their values (why are some values missing?).

In [ ]:
def prepare_data(
        boundary_vals, 
        initial_vals, 
        schrodinger_vals,
        num_col_train = 20000,
    ):
    """
    Create training and test data for training procedure.

    This function generates training and test datasets using a subset
    of the boundary values, initial values and the schrodinger values.
    Therefore the following steps need to be done:
        1. Data Point Selection
        2. Training Data Creation
        3. Testing Data Creation
        4. Unpacking Data
        5. Gradient Data 
        6. Dataset Creation
    
    Parameters
    ----------
    boundary_vals : Schrodinger_Boundary 
        Object containing boundary values.
    initial_vals : Schrodinger_Initial
        Object containing initial values.
    schrodinger_vals : Schrodinger
        Object containing schrodinger values.
    num_col_train : int
        Number of collocation points.
        
    Returns
    -------
    train_ds : Dict
        A dictionary containing "boundary", "initial" and "schrodinger" entries for training data.
    test_ds : Dict
        A dictionary containing "boundary", "initial" and "schrodinger" entries for testing data.
    """
    
    # Step 1: Data Point Selection
    num_col_tot = 20000
    all_idx = torch.arange(0, num_col_tot, dtype=torch.int)
    idx_col_train = torch.linspace(0, num_col_tot-1, num_col_train, dtype=torch.int)
    mask = ~torch.isin(all_idx, idx_col_train)
    idx_col_test = all_idx[mask]
    
    # Step 2: Training Data Creation
    schrodinger_train = schrodinger_vals[idx_col_train]
    schrodinger_train_x = schrodinger_train[0].clone().detach()
    schrodinger_train_t = schrodinger_train[1].clone().detach()
    
    # Step 3: Testing Data Creation
    schrodinger_test = schrodinger_vals[idx_col_test]
    schrodinger_test_x = schrodinger_test[0].clone().detach()
    schrodinger_test_t = schrodinger_test[1].clone().detach()

    # Step 4: Unpacking Data
    boundary_input_x = boundary_vals.getall()[0].clone().detach()
    boundary_input_t = boundary_vals.getall()[1].clone().detach()
    initial_input_x = initial_vals.getall()[0].clone().detach()
    initial_input_t = initial_vals.getall()[1].clone().detach()
    initial_target = initial_vals.getall()[2].clone().detach()

    # Step 5: Gradient Data 
    boundary_input_x.requires_grad = True
    boundary_input_t.requires_grad = True
    initial_input_x.requires_grad = True
    initial_input_t.requires_grad = True
    initial_target.requires_grad = True

    schrodinger_train_x.requires_grad = True
    schrodinger_train_t.requires_grad = True
    schrodinger_test_x.requires_grad = True  
    schrodinger_test_t.requires_grad = True

    boundary_inputs = [boundary_input_x, boundary_input_t]
    initial_inputs = [initial_input_x, initial_input_t]
    schrodinger_train = [schrodinger_train_x, schrodinger_train_t]
    schrodinger_test = [schrodinger_test_x, schrodinger_test_t]
    
    # Step 6: Dataset Creation
    train_ds = {"boundary": {"inputs": boundary_inputs, "targets": []}, 
                "initial": {"inputs": initial_inputs, "targets": initial_target},
                 "schrodinger": {"inputs": schrodinger_train, "targets": []}}
    test_ds = {"boundary": {"inputs": boundary_inputs, "targets": []}, 
                "initial": {"inputs": initial_inputs, "targets": initial_target},
                 "schrodinger": {"inputs": schrodinger_test, "targets": []}}

    # Return the created datasets
    return train_ds, test_ds


In [ ]:
train_ds, test_ds = prepare_data(boundary, initial, schrodinger, num_col_train=15000)

# Create PINN model

__Task:__  
Review the following class and take notes.  
Pay particular attention to:
- The number and width of layers.
- the forward pass.
- What input and output nodes the model posesses.

In [ ]:
class PINN(nn.Module):
    """
    Physics-Informed Neural Network (PINN) Class
    
    This class as a subclass of torch.nn.Module defines the architecture of the PINN model. 
    It is designed to use differential equations while incorporating physics-based constraints.
    The process consists of the following steps:
    
    Step 1: Model Initialization
        - Initialize the PINN model as a subclass of nn.Module.

    Step 2: Constructor Definition
        - Build a constructor to configure the model's architecture.
        - Utilize the nn.Linear class from the PyTorch library for defining layers and connections.
        - Initialize the weights of the layers using the Xavier uniform initializer.

    Step 3: Forward Pass Mechanism
        - Define the forward pass mechanism for the model, where input data flows through the layers
          to produce predicted outputs.
       
    """
    def __init__(self):
        """
        Constructor for the PINN class.
        
        Initializes the layers of the neural network:
        - Input layer fc1 taking a tensor with time and space data.
        - Four hidden fully connected layers fc2-fc5 with 50 neurons.
        - Output layer h_real for predicting the real part.
        - Output layer h_imag for predicting the imaginary part.
        
        Parameters
        ----------
        None
            
        Attributes
        ----------
        fc1 : nn.Linear
            First fully connected layer.
        fc2 : nn.Linear
            Second fully connected layer.
        fc3 : nn.Linear
            Third fully connected layer.
        fc4 : nn.Linear
            Fourth fully connected layer.
        fc5 : nn.Linear
            Fifth fully connected layer.
        h_real : nn.Linear
            Output layer for x-coordinate prediction.
        h_imag : nn.Linear
            Output layer for y-coordinate prediction.
        
        Returns
        -------
        None
        
        """
        super(PINN, self).__init__()
        layer_width = 50
        # Step 2: Constructor Definition
        torch.manual_seed(42)  # Set seed for reproducibility
        self.fc1 = nn.Linear(2,layer_width).to(device)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.fc2 = nn.Linear(layer_width,layer_width).to(device)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        self.fc3 = nn.Linear(layer_width,layer_width).to(device)
        torch.nn.init.xavier_uniform_(self.fc3.weight)
        self.fc4 = nn.Linear(layer_width,layer_width).to(device)
        torch.nn.init.xavier_uniform_(self.fc4.weight)
        self.fc5 = nn.Linear(layer_width,layer_width).to(device)
        torch.nn.init.xavier_uniform_(self.fc5.weight)
        self.h_real = nn.Linear(layer_width,1).to(device)
        torch.nn.init.xavier_uniform_(self.h_real.weight)
        self.h_imag = nn.Linear(layer_width,1).to(device)
        torch.nn.init.xavier_uniform_(self.h_imag.weight)
        pass
    
    def forward(self, x):
        """
        Perform a forward pass through the PINN model.
        
        This method defines the forward pass mechanism of the PINN model, where
        the input data X is processed through the layers to produce predicted
        outputs for both real part of the solution (h_real) 
        and imaginary part of the solution (h_imag).
        
        Parameters
        ----------
        t : torch.Tensor
            Input data tensor.
            
        Returns
        -------
        h_real : torch.Tensor
            Predicted real part of the solution.
        h_imag : torch.Tensor
            Predicted imaginary part of the solution.
        """

        # Step 1: Forward Pass Mechanism
        x = torch.tanh(self.fc1(x))   
        x = torch.tanh(self.fc2(x))  
        x = torch.tanh(self.fc3(x))  
        x = torch.tanh(self.fc4(x))  
        x = torch.tanh(self.fc5(x))     

        # Step 2: Produce predicted real and imaginary solution using output layers and return them. 
        h_real = self.h_real(x)
        h_imag = self.h_imag(x)

        return h_real, h_imag

# Boundary loss
To solve the time-dependent Schrödinger Equation, we need to consider the symmetry of the time evolution. This requires adding boundary points and defining a boundary loss that constrain the symmetry.  
Boundary Conditions:

*   $h(5,t)=h(-5,t) \quad \Leftrightarrow\quad  h(5,t)-h(-5,t)=0$ 

*   $\frac{\partial }{\partial x}h(5,t)=-\frac{\partial }{\partial x}h(-5,t) \quad\Leftrightarrow\quad \frac{\partial }{\partial x}h(5,t)+\frac{\partial }{\partial x}h(-5,t)=0$

Task:

Define the `compute_boundary_loss` function by following the steps stated in the docstring.

In [ ]:
def compute_boundary_loss(model : nn.Module, dataset : dict):
    """
    Define the boundary loss, representing one part of the physics-informed loss for the PINN model.
    
    This function calculates the loss used to incorporate the boundary conditions
    into the PINN. The following steps are involved:
    
    Step 1: Data Preparation
        - Get the boundary inputs (x_bound and t_bound) from the dataset.

    Step 2: Model Prediction
        - Concatenate the two input tensors along axis 1.
        - Using the concatenated tensors, predict the real and imaginary values with the neural network model.

    Step 3: Gradient Computation
        - Compute the first gradients 'hp_r_x', 'hp_i_x', 'hn_r_x', and 'hn_i_x' 
          with respect to the 'x' input tensor using the 'torch.autograd.grad' method.

    Step 4: Difference Calculation
        - Calculate the difference between 'hp' (h at x=5) and 'hn' (h at x=-5) for real and imaginary parts.

    Step 5: Derivative Sum Calculation
        - Calculate the Sum of 'hp_x' and 'hn_x' for real and imaginary parts.

    Step 6: Residual Definition
        - Define 'h_err' and 'h_x_err' by adding the squared difference/sum previously calculated.

    Step 7: Loss Calculation
        - Compute the mean squared error loss for 'h_err' and 'h_x_err' using the 'nn.MSELoss' class.
        - as a target serves a tensor containing zeros. It can be produced using 'torch.zeros_like'
        - Return the combined physics-informed loss as the sum of 'loss_h' and 'loss_h_x'.
    
    Parameters
    ----------
    model : torch.nn.Module
        The physics-informed neural network model.
    dataset : dict
        A dictionary containing "boundary" entries.
        
    Returns
    -------
    torch.Tensor
        The combined (h and h_x) physics-informed loss.
    """
    # Step 1: Data Preparation
    x_bound, t_bound = dataset["boundary"]["inputs"]

    
    # Step 2: Model Prediction
    hp_r, hp_i = model(torch.cat((x_bound, t_bound),1))
    hn_r, hn_i = model(torch.cat((-x_bound, t_bound),1))

    
    # Step 3: Gradient Computation
    hp_r_x = grad(outputs=hp_r, inputs=x_bound, grad_outputs=torch.ones_like(hp_r), create_graph=True)[0]
    # hp_i_x = 

    hn_r_x = grad(outputs=hn_r, inputs=x_bound, grad_outputs=torch.ones_like(hn_r), create_graph=True)[0]
    # hn_i_x = 

    
    # Step 4: Difference Calculation


    # Step 5: Derivative Sum Calculation


    # Step 6: Residual Definition


    # Step 7: Loss Calculation

    pass


__Checkpoint:__  
In the next cell, you will test your defined loss function on a generated dataset and an instance of the PINN.
If implemented correctly, it will always produce the same value, regardless of the randomly generated data points and network parameters.
This is due to the predefined seed value used for parameter and data initialization, which ensures consistent distribution every time.  

If you run the following cell and you don't produce an error, it means that your `compute_boundary_loss` function has been implemented correctly.

In [ ]:
Dummy_NN = PINN()

dummy_train_ds, dummy_test_ds = prepare_data(boundary, initial, schrodinger, num_col_train=20000)

dummy_boundry_loss = compute_boundary_loss(Dummy_NN, dummy_train_ds)

assert round(dummy_boundry_loss.item(), 5) == 0.07453, f'Your boundary loss of {round(dummy_boundry_loss.item(), 5)} is not correct!'


# Initial loss

This part is used, to implement the initial state $\psi(x,0)$ of the system, as visualized above.  
__Task:__  
Define the `compute_initial_loss` function by following the steps stated in the docstring.

In [ ]:
def compute_initial_loss(model : nn.Module, dataset: dict):
    """
    Calculate the loss associated with satisfying the initial conditions for the PINN model.

    This function computes the loss that measures how well the initial conditions are met by the PINN. 
    The dataset containes information about the initial condition.
    The following steps are involved:

    Step 1: Data Preparation
        - Extract 'x_0', 't_0' and 'psi_0' from the training dataset.

    Step 2: Model Prediction
        - Concatenate the two input tensors along axis 1.
        - Use the neural network model to predict the real and imaginary values at (x_0, t_0).

    Step 3: Concatenation
        - Concatenate the real and imaginary parts of the predicted values along axis 1.

    Step 4: Loss Calculation
        - Calculate and return the mean squared error loss between 'psi_0' and the concatenated values 'h'.

    Parameters
    ----------
    model : torch.nn.Module
        The physics-informed neural network model.
    dataset : dict
        A dictionary that includes "initial" entries with "x_0," "t_0," and "psi_0."

    Returns
    -------
    torch.Tensor
        The loss quantifying how well the initial conditions are satisfied.
    """
    # Step 1: Data Preparation

    # Step 2: Model Prediction

    # Step 3: Concatenation
    
    # Step 4: Loss Calculation

    pass

__Checkpoint:__  
If you run the following cell and you don't produce an error, it means that your ``compute_initial_loss`` function has been implemented correctly.

In [ ]:
dummy_initial_loss = compute_initial_loss(Dummy_NN, dummy_train_ds)

assert round(dummy_initial_loss.item(), 5) == 0.04446, f'Your initial loss {round(dummy_initial_loss.item(), 5)} is not correct!'

# Physics-informed Loss

The time dependant Schrodinger Equation describes the time evolution of the quantum mechanic harmonic oscillator:

$i\hbar\frac{\partial}{\partial t}\psi(x,t)=\hat{H} \psi(x,t) $  

After reordering and using the Hamiltonian of the quantum harmonic oscillator, we obtain:

$\left(i\hbar\frac{\partial}{\partial t}+\frac{\hbar^2}{2m}\frac{\partial^2}{\partial x^2}-\frac{m \omega^2}{2}x^2\right) \psi(x,t)=0$ 

However, this equation cannot be directly applied to the `compute_physics_informed_loss` function in its current form.  
This is because the model predicts separate solutions for the real and imaginary parts, labeled as $\psi_r(x,t) $ and $ \psi_i(x,t)$ respectively, instead of a single solution $\psi(x,t)$.  
To implement the equation, first divide it into real and imaginary parts.  
Complete the following tasks:

1. Replace $\psi(x,t)$ with $\psi_r(x,t) + i \psi_i(x,t)$ and insert it into the equation.

2. Split the left side of the equation into its real and imaginary parts.

3. Follow the step-by-step explanation to define the `compute_physics_informed_loss` function and use the  
   results from Task 2 to define the residuals `f_r` and `f_i`.  
   __hint__: Set $ m =1 $, $\hbar =1$ and $\omega =1$  

Comment: The boolean value `potential_scaling` is used in the final task, so ignore it for now.




In [ ]:
def compute_physics_informed_loss(model : nn.Module, dataset: dict, potential_scaling:bool = False):
    """
    Calculate the loss for the Schrödinger equation condition in the PINN model.

    This function computes the loss that measures how well the Schrödinger equation condition
    is satisfied by the PINN. The following steps are involved:

    Step 1: Data Preparation
        - Get the schrodinger inputs (x_schr and t_schr) from the dataset.

    Step 2: Model Prediction
        - Concatenate the two input tensors along axis 1.
        - Use the neural network model to predict the real and imaginary values.

    Step 3: Derivative Computation
        - Compute the first derivatives with respect to 't_schr' and the second derivatives with respect to 'x_schr' 
          for both the real and the imaginary part of the solution.

    Step 4: Residuals Definition
        - Define residuals 'f_r' and 'f_i' that contain the Schrödinger equation condition for the real and imaginary parts.

    Step 5: Residual Calculation
        - Define a residual 'f' by computing the Euclidean norm from 'f_r' and 'f_i.'

    Step 6: Loss Calculation
        - Calculate and return the mean squared error loss between 'f' and zeros.

    Parameters
    ----------
    model : torch.nn.Module
        The physics-informed neural network model.
    dataset : dict
        A dictionary containing "schrodinger" entries.
    potential_scaling : bool
        Whether the potential scaling is set to True of false.
        
    Returns
    -------
    torch.Tensor
        The loss quantifying how well the Schrödinger equation condition is satisfied.
    """
    # Step 1: Data Preparation

    # Step 2: Model Prediction

    # Step 3: Derivative Computation

    # Step 4: Residuals Definition

    # Step 5: Residual Calculation

    # Step 6: Loss Calculation
    
    pass

__Checkpoint:__  
If you run the following cell and you don't produce an error, it means that your ``compute_physics_informed_loss`` function has been implemented correctly.

In [ ]:
dummy_physics_loss = compute_physics_informed_loss(Dummy_NN, dummy_train_ds, potential_scaling=False)

assert round(dummy_physics_loss.item(), 5) == 5.61577, f'The physics loss {round(dummy_physics_loss.item(), 5)} is not correct!'

# Total loss

In [ ]:
def compute_total_loss(model : nn.Module, dataset: dict, potential_scaling:bool):
    """
    Define the total loss for the physics-informed neural network.

    This function computes the total loss for the PINN model by combining three
    different components:  boundary, inital and physics-informed loss.
    The following steps are involved:

    Step 1: Loss Calculation
        - Determine the boundary, inital and physics loss using the corresponding functions.

    Step 2: Total Loss Combination
        - Return the combined total loss as the sum of the three losses.


    Parameters
    ----------
    model : torch.nn.Module
        The physics-informed neural network model.
    dataset : dict
        A dictionary containing "inputs", "targets_x", "targets_y", and "t_col" entries.

    Returns
    -------
    torch.Tensor
        The combined total loss considering data and physics constraints.
    """
    # Step 1: Loss Calculation
    initial_loss = compute_initial_loss(model, dataset)
    boundary_loss = compute_boundary_loss(model, dataset)
    physics_loss = compute_physics_informed_loss(model, dataset, potential_scaling)

    # Step 2: Total Loss Combination
    total_loss = initial_loss + physics_loss + boundary_loss

    return total_loss


__Checkpoint:__  
If you run the following cell and you don't produce an error, it means that your ``compute_total_loss`` function has been implemented correctly.

In [ ]:
dummy_total_loss = compute_total_loss(Dummy_NN, dummy_train_ds, potential_scaling=False)

assert round(dummy_total_loss.item(), 5) == 5.73475, f'The total loss {round(dummy_total_loss.item(), 5)} is not correct!'

# Execution function

__Task:__  
Inspect the execute function and take notes.

In [ ]:
def execute(
        model : nn.Module,
        train_ds: dict,
        test_ds: dict,
        lr = 0.00025, 
        num_epochs = 2500,
        num_batches = 25,
        potential_scaling = False,
    ):
    """
    Execute the training procedure for a physics-informed neural network model.

    This function trains the model using specified hyperparameters and returns relevant data.

    
    Parameters
    ----------
    model : torch.nn.Module
        The neural network model to be trained and evaluated.
    train_ds : dict
        A dictionary containing initial, boundary and schrodinger training data.
    test_ds : dict
        A dictionary containing initial, boundary and schrodinger testing data.
    lr : float, optional
        Learning rate for the optimizer. Default is 0.00025.
    num_epochs : int, optional
        Number of epochs to train the model. Default is 2000.
    num_batches : int, optional
        Number of batches to split the training data into. Default is 25.
    potential_scaling : bool, optional
        Whether to use the potential scaling or not. Default is False.


    Returns
    -------
    train_loss_evolution : list
        A list containing train loss values during training.
    test_loss_evolution : list
        A list containing test loss values during training
    predictions_list : list
        A list containing the model predictions at logarithmically sampled epochs.
    """   

    # Step 1: Optimizer Initialization
    optimizer = optim.Adam(model.parameters(), lr=lr,)  # Include weight_decay in the optimizer

    # Step 2: Lists Initialization
    train_loss_evolution = []
    test_loss_evolution = []
    predictions_list = []

    # Define Loading Bar
    loading_bar = trange(1, num_epochs + 1)

    # Step 3: Training Loop Setup
    for epoch in loading_bar:
        # Step 4: Epoch Loss Tracking
        train_loss = compute_total_loss(model, train_ds, potential_scaling)
        test_loss = compute_total_loss(model, test_ds, potential_scaling)
        test_loss_evolution.append(float(test_loss))
        train_loss_evolution.append(float(train_loss))

        # Prediction tracking
        if epoch in logarithmic_sampling(num_epochs):
            h_hat = get_prediction(model, period=shared_data.period, device=device)
            predictions_list.append(h_hat)

        # Step 5: Permutation of Training Data
        permutations = torch.randperm(min(train_ds["schrodinger"]["inputs"][0].shape[0], 
                                        train_ds["initial"]["inputs"][0].shape[0], 
                                        train_ds["boundary"]["inputs"][0].shape[0]))
        permutations = torch.tensor_split(permutations, num_batches)

        # Step 6: Nested Batch Training Loop Setup
        for i in range(len(permutations)):
            # Step 7: Data Preparation
            train_batch_ds = {"boundary": {"inputs": [], "targets": []}, 
                            "initial": {"inputs": [], "targets": []},
                            "schrodinger": {"inputs": [], "targets": []}}

            perm_init_x = torch.tensor_split(train_ds["initial"]["inputs"][0], num_batches)
            perm_init_t = torch.tensor_split(train_ds["initial"]["inputs"][1], num_batches)
            train_batch_ds["initial"]["inputs"] = [perm_init_x[i], perm_init_t[i]]
            
            perm_init_psi = torch.tensor_split(train_ds["initial"]["targets"], num_batches)
            train_batch_ds["initial"]["targets"] = perm_init_psi[i]
        
            perm_bound_x = torch.tensor_split(train_ds["boundary"]["inputs"][0], num_batches)
            perm_bound_t = torch.tensor_split(train_ds["boundary"]["inputs"][1], num_batches)
            train_batch_ds["boundary"]["inputs"] = [perm_bound_x[i], perm_bound_t[i]]
        
            perm_schrodinger_x = torch.tensor_split(train_ds["schrodinger"]["inputs"][0], num_batches)
            perm_schrodinger_t = torch.tensor_split(train_ds["schrodinger"]["inputs"][1], num_batches)
            train_batch_ds["schrodinger"]["inputs"] = [perm_schrodinger_x[i], perm_schrodinger_t[i]]

        
            # Step 8: Batch Training
            optimizer.zero_grad()
            train_loss = compute_total_loss(model, train_batch_ds, potential_scaling)
            train_loss.backward()
            optimizer.step()

        # Step 9: Print Progress
        loading_bar.set_description(f"Epoch: {epoch}")
        loading_bar.set_postfix({"Test Loss": test_loss.item(), "Train Loss": train_loss.item()})
        

    # Step 10: Data Return
    return train_loss_evolution, test_loss_evolution, predictions_list

# Execution task: Verification
__Task:__  
1. Initialize the model by calling the PINN class created earlier.
2. Generate training and test datasets by calling the 'prepare_data' function.
    - Use __20000 collocation training points__ for data generation.
3. Execute the training for the following set of (hyper)parameters:        
    - Train over  $n_\text{epochs} =2500$ __epochs__.
    - Set the number of __mini-batches__ to $n_\text{batches}=25$
    - Set `potential_scaling` to __False__

The training requires some time. Take a coffee break and do some push ups.  
After completion, check the loss evolution by running the next cell.
A loss around $\mathcal{L}=10^{-4}$ indicates good performance.

In [ ]:
# Step 1: Initialize the model by calling the PINN class created earlier.
model = PINN()

# Step 2: Generate training and test datasets by calling the 'prepare_data' function.
train_ds, test_ds = prepare_data(boundary, 
        initial, 
        schrodinger,
        num_col_train=20000,
        )
# Step 3: Execute the training and testing of the model.
train_loss_evolution, test_loss_evolution, predictions_list = execute(
    model,
    test_ds=test_ds,
    train_ds=train_ds,
    num_epochs=2500,
    num_batches=25,
    potential_scaling=False,
)

__Saving the trained model__:  
By running the following cell, the trained model and its adjusted parameters will be saved in a file named `trained_model.pth`. 
If you need to restart your kernel for any reason, you can load the trained model by running the subsequent cell.
Ensure that you run the import cells and the cell where the PINN class is defined.

In [ ]:
# Save the model
torch.save(model, "trained_model.pth")

In [ ]:
# Now you can load the model
loaded_model = torch.load("trained_model.pth", weights_only=False)

In [ ]:
visualize_training(train_loss_evolution, test_loss_evolution)

# Result animation

After training the model, it is time to verify that it has learned the time evolution of the initial state.  

__Tasks:__
- To produce and export four animations as mp4-files, as well as the last frame of the animations as svg-files, run the following cell.
- Investigate the animations and determine their meaning. their meaning and present your findings, along with the animations, to your tutor for approval before proceeding.

In [ ]:
animator = AnimationCreator(model, period, predictions_list=predictions_list, device=device)
animator.create_meshgrid()
animator.compute_solution()
animator.create_train_evolution_real_animation()
animator.create_train_evolution_imag_animation()
animator.create_cross_section_animation()
animator.create_complex_plot_animation()

# Execution task 4: Energy Eigenvalues
The PINN model can now predict the temporal evolution of the wave function over a period.  
We will now test whether the energy of the corresponding predicted wave function is consistent with the theory.  
To accomplish this, we begin with the eigenvalue equation of the Hamiltonian.  <br>

$\hat{H}| \psi_n \rangle = E_n  | \psi_n \rangle$  

We multiply both sides with the Bra vector $\langle \psi_n |$ to it:  

$\langle \psi_n| \hat{H} | \psi_n \rangle = E_n \langle \psi_n | \psi_n \rangle$  

We can now evaluate the integral in position space and solve for the energy eigenvalue.  
Let's also denote both integrals with $I_1$ and $I_2$, so that we can treat them each at a time:

$E_n = \underbrace{\left(\int_{-\infty}^{\infty} \psi_n^*(x,t)\cdot \hat{H}\cdot \psi_n(x,t) \,dx\right)}_{I_1}/\underbrace{\left(\int_{-\infty}^{\infty}\psi_n^*(x,t)\cdot \psi_n(x,t)\,dx\right)}_{I_2}$

Writing out the Hamiltonian in integral $I_1$ we obtain:

$I_1=\int_{-\infty}^{\infty} \psi_n^*(x,t)\cdot\left(\frac{\hat{p}^2}{2m}+\frac{m \omega^2}{2}x^2\right)\cdot \psi_n(x,t) \,dx$

The integral can be divided into two parts, named $I_{1a}$ and $I_{1b}$.

$I_1=\underbrace{\frac{1}{2m}\int_{-\infty}^{\infty} \psi_n^*(x,t)\cdot\hat{p}^2\psi_n(x,t)\,dx}_{I_{1a}} + \underbrace{\frac{m \omega^2}{2}\int_{-\infty}^{\infty} \psi_n^*(x,t)\cdot\psi_n(x,t)\cdot x^2 \,dx}_{I_{1b}}$

Integral $I_{1b}$ can be further simplified, since $\psi_n^*(x,t)\cdot\psi_n(x,t)=|\psi_n(x,t)|^2$:

$I_{1b}=\frac{m \omega^2}{2}\int_{-\infty}^{\infty} |\psi_n(x,t)|^2\cdot x^2 \,dx$

After reordering and combining everything, we end up with:

$E_n = \frac{\overbrace{\frac{m \omega^2}{2}\int_{-\infty}^{\infty} |\psi_n(x,t)|^2\cdot x^2 \,dx}^{I_{1b}} + \overbrace{\frac{1}{2m}\int_{-\infty}^{\infty} \psi_n^*(x,t)\cdot\hat{p}^2\psi_n(x,t) \,dx}^{I_{1a}} }{\underbrace{\int_{-\infty}^{\infty}|\psi_n(x,t)|^2,dx}_{I_2}}$
___
4. Energy Eigenvalues:  

   However, we cannot directly apply this equation to our model in this form.  
   Because, the model prediction is limited to the $[-5,5]$ in space.  
   Additionally, the model provides separate solutions for the real and imaginary parts of $\psi$, rather than a single solution.  
   Therefore complete the following Tasks: 
   <br><br>

   1. Split up the right side of the last equation, into a real and an imaginary part.  
      __Hint__: Integral $I_{1b}$ and $I_2$ are already purely real, so don't waste your time on them.
   <br><br>

   2. Implement a function called `compute_momentum_squared`  
      that returns the negative second derivative of the real and imaginary parts of the model's prediction.
      <br><br>

   3. Implement a function called `compute_energy_eigenvalue`  
      that returns a list containing a number of `n_time_steps` Energy values between `t=0` and `t=period`.  
      __hint__: Set $ m =1 $, $\hbar =1$ and $\omega =1$  
      __hint__: To calculate an integral, use the torch command `torch.trapz`.
      <br><br>

   4. Plot the real and imaginary parts of the energy eigenvalue over time  
      and explain the result in the evaluation.


In [ ]:
def compute_momentum_squared(model : nn.Module, x, t):
    """
    Calculate the momentum squared of the wavefunction.

    This function computes the momentum squared of the wavefunction by using the
    neural network model.

    Parameters
    ----------
    model : torch.nn.Module
        The physics-informed neural network model.
    x : torch.Tensor
        The x values.
    t : torch.Tensor
        The t values.
    
    Returns
    -------
    p2_h_r : torch.Tensor
        The momentum squared of the real part wavefunction.
    p2_h_i : torch.Tensor
        The momentum squared of the imaginary part wavefunction.
    """
    pass

In [ ]:
def compute_energy_eigenvalue(model : nn.Module, n_time_steps=100):
    """
    This function computes the energy eigenvalues of the Schrödinger equation.

    parameters
    ----------
    model : torch.nn.Module
        The physics-informed neural network model.
    n_time_steps : int
        The number of time steps.
    
    Returns
    -------
    energy_eigenvalues_real : list
        The real part of the energy eigenvalues.
    energy_eigenvalues_imag : list
        The imaginary part of the energy eigenvalues.
    """
    # Step 1: Data Preparation
    x = torch.linspace(-5,5,1000)
    t = torch.ones_like(x)

    # Step 2: Gradient Requirement
    x.requires_grad = True
    t.requires_grad = True

    # Step 3: List Initialization
    energy_eigenvalues_real = []
    energy_eigenvalues_imag = []


    for i in range(n_time_steps):
        # Step 4: time Initialization
        time = i/n_time_steps*period
        t = torch.ones_like(x)*time

        # Step 5: Model Prediction
        p2_h_r, p2_h_i = compute_momentum_squared(model, x, t)
        h_r, h_i = model(torch.cat((x.unsqueeze(1), t.unsqueeze(1)),1))

        # Step 6: Integral Calculation
        inside_integral_2= (h_r**2 + h_i**2).squeeze(1)
        integral_2 = torch.trapz(inside_integral_2, x, dim=0)

        inside_integral_1b_real = (h_r**2 +h_i**2).squeeze(1) * x**2/2
        # inside_integral_1a_real =                                         # FILL IN
        # inside_integral_1_real =                                          # FILL IN
        # integral_1_real =                                                 # FILL IN    
        energy_eigenvalue_real = integral_1_real/integral_2

        # inside_integral_1a_imag =                                         # FILL IN
        # integral_1_imag =                                                 # FILL IN
        energy_eigenvalue_imag = integral_1_imag/integral_2

        # Step 7: Array Filling
        energy_eigenvalues_real.append(energy_eigenvalue_real.detach().numpy().astype(float))
        energy_eigenvalues_imag.append(energy_eigenvalue_imag.detach().numpy().astype(float))

    return energy_eigenvalues_real, energy_eigenvalues_imag



In [ ]:
def visualize_energy_eigenvalue(model : nn.Module):
    """
    Visualize the energy eigenvalues of the Schrödinger equation.

    This function visualizes the energy eigenvalues of the Schrödinger equation by using the
    neural network model.

    Parameters
    ----------
    model : torch.nn.Module
        The physics-informed neural network model.
    
    Returns
    -------
    None
    """
    energy_eigenvalues_real, energy_eigenvalues_imag = compute_energy_eigenvalue(model)
    t = np.linspace(0, period, len(energy_eigenvalues_real))
    fig, ax1 = plt.subplots()

    # Plotting real part on the left side
    line1, = ax1.plot(t, energy_eigenvalues_real, label='Real part model prediction', marker='x', linestyle='None', color='tab:blue')
    line2, = ax1.plot(t, 0.5 * np.ones_like(t), label='Real part true solution', color='tab:blue')
    ax1.set_xlabel(r"$t\,[\omega^{-1}]$", fontsize=16)
    ax1.set_ylabel(rf'$E_{shared_data.n}^r\,[\hbar\omega]$', fontsize=16)
    ax1.tick_params(axis='y', labelsize=14)
    ax1.legend(loc='upper left', fontsize=16)


    # Creating a twin Axes for the imaginary part on the right side
    ax2 = ax1.twinx()
    line3, = ax2.plot(t, energy_eigenvalues_imag, label='Imaginary part model prediction', marker='+', linestyle='None', color='tab:red')
    line4, = ax2.plot(t, np.zeros_like(t), label='Imaginary part true solution', linestyle='--', color='tab:red')
    ax2.set_ylabel(rf'$E_{shared_data.n}^i\,[\hbar\omega]$', fontsize=16, )#color='tab:red')
    ax2.tick_params(axis='y', labelsize=14)

    # Combine lines and labels for a single legend
    lines = [line1, line2, line3, line4]
    labels = [line.get_label() for line in lines]

    # Placing a single legend outside the frame
    ax1.legend(lines, labels, loc='upper left', fontsize=16, bbox_to_anchor=(1.2, 0.67), borderaxespad=0.)

    # Set scalar formatter with increased font size for y-axis ticks
    ax2.yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
    ax2.yaxis.get_major_formatter().set_powerlimits((0, 1))
    ax2.yaxis.offsetText.set_fontsize(14)

    plt.show()
    fig.savefig(f'Energie_eigenvalue_{shared_data.n}.svg', format='svg', bbox_inches='tight')

In [ ]:
visualize_energy_eigenvalue(model)

# Execution task 5: Diminishing Potential 
5. __Diminishing Potential__  
  
    All the previous tasks could have been solved by hand using a time-evolution operator. 
    Using the PINN, however, we have a general framework that can solve the time evolution for any potential.
    For the last step of this course, we will therefore study a non-trivial example of a time-evolving potential:
    We start with the harmonic potential and multiply it with a time-dependent factor that decreases from 1 to 0.  

    The implementation of this diminishing potential is done in 3 steps:  

    1. Implement a function called `potential_scaling_function` that behaves like: $
        A(t) = 
        \begin{cases} 
            \cos^2(\frac{t}{(period)/\pi}) & \text{if } t < \frac{\text{{period}}}{2} \\
            0 & \text{otherwise}
        \end{cases}
        $
    <br><br>

    2. Implement the function at the right spot into the `compute_physics_informed_loss` function.  
    <br><br>

    3. Execute the training using the same settings as before, but limit the number of epochs to `num_epochs`$=300$.  
       Plot the result using the `SolutionVisualizer` class and save the contour plots for an evaluation.  
       ___IMPORTANT:___  
       __Don't__ use the `AnimationCreator`, it would overwrite your previous results.

In [ ]:
def potential_scaling_function(t:torch.Tensor, potential_scaling:bool)->torch.Tensor:
    """
    This function defines the potential scaling function.
    
    Parameters
    ----------
    t : torch.Tensor
        The time values.
    potential_scaling : bool
        Whether the potential scaling is set to True of false.
    
    Returns
    -------
    torch.Tensor
        The potential scaling function.
    """
    pass

In [ ]:
def visualize_potential_scaling_function(potential_scaling):
    """
    This function visualizes the potential scaling function.
    
    Parameters
    ----------
    potential_scaling : bool
        Whether the potential scaling is set to True of false.
    
    Returns
    -------
    None
    """
    period = shared_data.period
    t_tensor = torch.linspace(0, period, 100)
    potential_scaling_result = potential_scaling_function(t_tensor, potential_scaling=potential_scaling)
    %matplotlib inline
    fig, ax = plt.subplots()

    ax.plot(t_tensor, potential_scaling_result, label='Potential scaling')

    ax.set_xlabel(r"$t\,[\omega^{-1}]$", fontsize=16)
    ax.set_ylabel(r'$A(t)$', fontsize=16)
    #ax.legend()

    plt.show()
    fig.savefig('Potential_scaling.svg', format='svg', bbox_inches='tight')


In [ ]:
visualize_potential_scaling_function(potential_scaling=True)

In [ ]:
vis = SolutionVisualizer(model=model, period=shared_data.period, potential_scaling=True, n=0, save_svg=True)
vis.visualize()